In [35]:
import numpy as np
import open3d as o3d
from collections import deque

def visualize_pcd(pcd):
    o3d.visualization.draw_geometries(pcd,
                                    zoom=0.49,
                                    front=[-0.4999, -0.1659, -0.8499],
                                    lookat=[2.1813, 2.0619, 2.0999],
                                    up=[0.1204, -0.9852, 0.1215])

test_data_path = "wall_table_chair.pcd"

pcd = o3d.io.read_point_cloud(test_data_path)

box_points = np.asarray(pcd.points)
print(f"Initial PC Size: {box_points.shape}")

box_points_green = deque()
box_points_red = deque()

# Split the mesh box points in two so that we draw only half of the points
for i in range(0, box_points.shape[0]):
    if i >= box_points.shape[0]/2:
        box_points_red.append(box_points[i])
    else:
        box_points_green.append((box_points[i]))

print(f"Red Box Points Size: {len(box_points_red)}")
print(f"Green Box Points Size: {len(box_points_green)}")

# Create two new objects that we'll assign half of the points
# and then paint in one color
cluster_1 = o3d.geometry.PointCloud()
cluster_1.points = o3d.utility.Vector3dVector(box_points_red)

cluster_2 = o3d.geometry.PointCloud()
cluster_2.points = o3d.utility.Vector3dVector(box_points_green)

cluster_1.paint_uniform_color([1.0, 0, 0])
cluster_2.paint_uniform_color([0.0, 0, 1.0])

visualize_pcd([cluster_1, cluster_2])








Initial PC Size: (40000, 3)
Red Box Points Size: 20000
Green Box Points Size: 20000
